<a href="https://colab.research.google.com/github/SattuSupCodes/SentimentAnalysis_TheoreticalLearning/blob/main/ECAPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ejlok1/cremad")

print("Path to dataset files:", path)

100%|██████████| 451M/451M [00:03<00:00, 151MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ejlok1/cremad/versions/1


# Pin Your Install Versions
when dealing with tools that are not compatible with latest upgrades, the smartest choice (coming from not the brightest star rn) is to pin your versions.

In [2]:
!pip install --no-cache-dir \
  "torch==2.0.1+cpu" \
  "torchaudio==2.0.2+cpu" \
  --index-url https://download.pytorch.org/whl/cpu

!pip install "speechbrain==0.5.16" soundfile

Looking in indexes: https://download.pytorch.org/whl/cpu
ERROR: Could not find a version that satisfies the requirement torch==2.0.1+cpu (from versions: 2.2.0+cpu, 2.2.1+cpu, 2.2.2+cpu, 2.3.0+cpu, 2.3.1+cpu, 2.4.0+cpu, 2.4.1+cpu, 2.5.0+cpu, 2.5.1+cpu, 2.6.0+cpu, 2.7.0+cpu, 2.7.1+cpu, 2.8.0+cpu, 2.9.0+cpu, 2.9.1+cpu)
ERROR: No matching distribution found for torch==2.0.1+cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 28.3 MB/s eta 0:00:00


In [15]:
import numpy as np
import librosa
import os
import torchaudio
from speechbrain.lobes.models.ECAPA_TDNN import ECAPA_TDNN
from speechbrain.pretrained import EncoderClassifier
import torch
from sklearn.cluster import AgglomerativeClustering


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu" # imagine not having cuda (heartbreak emoji)

ecapa = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    run_opts={"device": device}
)

ecapa.eval()
print("ECAPA loaded (frozen embedding extractor)")


hyperparams.yaml: 0.00B [00:00, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt: 0.00B [00:00, ?B/s]

ECAPA loaded (frozen embedding extractor)


We downloaded our PRETRAINED ecapa

In [9]:
audio_path = os.path.join(
    path, "AudioWAV", "1001_IEO_SAD_HI.wav"
)

Lets be kind to our technology. Diarisation is NOT per dataset, its pr audio file.


In [19]:
import soundfile as sf
import torch

wav, sr = sf.read("sample4.wav")

wav = torch.tensor(wav, dtype=torch.float32)

if wav.ndim == 2:   # stereo → mono meaning combining two channels in one because our ecapa can only process mono audios
    wav = wav.mean(dim=1)

print(wav.shape, sr)



torch.Size([4449280]) 44100


In [24]:
window_sec = 2.0
hop_sec = 1.0

window_samples = int(window_sec * sr)
hop_samples = int(hop_sec * sr)

segments = []

for start in range(0, len(wav) - window_samples, hop_samples):
    segment = wav[start:start + window_samples]
    segments.append(segment)

print(f"Segments created: {len(segments)}")


Segments created: 99


In [25]:
embeddings = []

with torch.no_grad():
    for seg in segments:
        seg = seg.unsqueeze(0).to(device)  # [1, T]
        emb = ecapa.encode_batch(seg)      # [1, 1, D]
        emb = emb.squeeze().cpu().numpy()
        embeddings.append(emb)

embeddings = np.vstack(embeddings)
print("Embeddings shape:", embeddings.shape)
# o/p (number of segments, embedding dimensions)-> speaker info in vector form

Embeddings shape: (99, 192)


In [26]:

clusterer = AgglomerativeClustering(
    n_clusters=None,                 # <-- key
    metric="cosine",
    linkage="average",
    distance_threshold=0.4            # tune this
)

labels = clusterer.fit_predict(embeddings)
print("Speakers detected:", len(set(labels)))


Speakers detected: 5


In [27]:
for i, label in enumerate(labels[:10]):
    start_time = i * hop_sec
    print(f"{start_time:.2f}s → Speaker {label}")


0.00s → Speaker 2
1.00s → Speaker 2
2.00s → Speaker 0
3.00s → Speaker 0
4.00s → Speaker 0
5.00s → Speaker 2
6.00s → Speaker 2
7.00s → Speaker 2
8.00s → Speaker 2
9.00s → Speaker 1
